## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,33.06,90,100,2.77,overcast clouds
1,1,Xining,CN,36.6167,101.7667,56.23,86,100,2.73,overcast clouds
2,2,Clyde River,CA,70.4692,-68.5914,39.43,87,75,9.22,broken clouds
3,3,Busselton,AU,-33.6500,115.3333,53.49,66,94,8.37,overcast clouds
4,4,San Rafael Del Sur,NI,11.8485,-86.4384,83.37,80,100,7.31,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,San Rafael Del Sur,NI,11.8485,-86.4384,83.37,80,100,7.31,overcast clouds
7,7,Puerto Ayora,EC,-0.7393,-90.3518,75.16,82,96,4.00,overcast clouds
13,13,Jamestown,US,42.0970,-79.2353,81.05,57,40,6.91,scattered clouds
17,17,Albany,US,42.6001,-73.9662,84.85,54,84,6.80,broken clouds
22,22,Port Elizabeth,ZA,-33.9180,25.5701,81.10,42,0,6.91,clear sky
23,23,Kruisfontein,ZA,-34.0033,24.7314,78.89,29,31,7.40,scattered clouds
27,27,Hithadhoo,MV,-0.6000,73.0833,81.36,73,100,5.10,overcast clouds
28,28,Lorengau,PG,-2.0226,147.2712,79.74,72,6,5.44,clear sky
30,30,Georgetown,MY,5.4112,100.3354,80.53,89,20,2.30,few clouds
33,33,Wanning,CN,18.8003,110.3967,76.37,90,100,7.76,moderate rain


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
clean_df = preferred_cities_df

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,San Rafael Del Sur,NI,83.37,overcast clouds,11.8485,-86.4384,
7,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
13,Jamestown,US,81.05,scattered clouds,42.0970,-79.2353,
17,Albany,US,84.85,broken clouds,42.6001,-73.9662,
22,Port Elizabeth,ZA,81.10,clear sky,-33.9180,25.5701,
23,Kruisfontein,ZA,78.89,scattered clouds,-34.0033,24.7314,
27,Hithadhoo,MV,81.36,overcast clouds,-0.6000,73.0833,
28,Lorengau,PG,79.74,clear sky,-2.0226,147.2712,
30,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
33,Wanning,CN,76.37,moderate rain,18.8003,110.3967,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name'] != '')]
clean_hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,San Rafael Del Sur,NI,83.37,overcast clouds,11.8485,-86.4384,Hospedaje El Chino
7,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Jamestown,US,81.05,scattered clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
22,Port Elizabeth,ZA,81.10,clear sky,-33.9180,25.5701,39 On Nile Guest House
23,Kruisfontein,ZA,78.89,scattered clouds,-34.0033,24.7314,Oyster Bay House Rental
27,Hithadhoo,MV,81.36,overcast clouds,-0.6000,73.0833,Scoop Guest House
28,Lorengau,PG,79.74,clear sky,-2.0226,147.2712,Lorengau Harbourside Hotel
30,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
33,Wanning,CN,76.37,moderate rain,18.8003,110.3967,Paradise Hotel
35,Lazaro Cardenas,MX,87.53,few clouds,17.9583,-102.2000,HOTEL QUINTA ANTIGUA


In [28]:
# Create the output file (CSV).
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))